# Final (Fine Dining)

In [1]:
import pandas as pd
import numpy as np
import datetime
import pandas as pd

In this final, you will be asked to analyze a data set containing historical reservations for a fine-dining restaurant that is stored in the file "data/Reservation_Data.csv".  This restaurant only accepts reservations booked online and hence does not accept walk in customers.   Each row in this data corresponds to a booked reservation.  The columns have the following meaning:
- reservation\_date: The date that the reservation was booked for. In other words, this is the date when the customer will dine.
- reservation\_time: The time that the reservation was booked for.
- reservation\_party\_size: The size of the party for the corresponding reservation, i.e. the number of diners.
- reservation_date_booked: the date on which the reservation was made.
- datetime\_booked: The date and time corresponding to when the reservation was made (in UTC). This column has missing values, which have been entered as "#N/A".  **The restaurant is located in a Pacific time zone.**

Your goal in this final will be to undertand how customers schedule reservations at this restaurant.


## Problem 1 

Your first task is to read in the data and do the following:

- delete rows with missing values
- convert datetime_booked to a datetime column with Pacific time zone
- combine reservation_date and reservation time to create a new column called reservation_datetime that is a datetime column that has a Pacific time zone. So the final dataframe you return should have 6 columns in total.
- Only keep reservations made at the following 8 time slots: 17:30, 17:45, 18:00, 18:15, 20:45, 21:00, 21:15, 21:30 (**Hint**: You need to use the reservation_datetime column to solve this).

Return this modified version of the original data frame.

In [2]:
def Read_Data():
    
    df_res = pd.read_csv("data/Reservation_Data.csv", na_values=["#N/A"])
    # YOUR CODE HERE
#     raise NotImplementedError()
    
    df_res.dropna(inplace=True)
    df_res['datetime_booked'] = pd.to_datetime(df_res['datetime_booked']).dt.tz_localize("GMT").dt.tz_convert('America/Los_Angeles').dt.tz_localize(None)
    df_res['reservation_datetime'] = df_res.apply(lambda x: x['reservation_date'] + ' '+ x['reservation_time'], axis=1)
    df_res['reservation_datetime'] = pd.to_datetime(df_res['reservation_datetime'])
    
    time_list = [
        datetime.datetime.strptime("17:30","%H:%M").time(),
        datetime.datetime.strptime("17:45","%H:%M").time(),
        datetime.datetime.strptime("18:00","%H:%M").time(),
        datetime.datetime.strptime("18:15","%H:%M").time(),
        datetime.datetime.strptime("20:45","%H:%M").time(),
        datetime.datetime.strptime("21:00","%H:%M").time(),
        datetime.datetime.strptime("21:15","%H:%M").time(),
        datetime.datetime.strptime("21:30","%H:%M").time(),
                ]
    df_res = df_res[df_res['reservation_datetime'].dt.time.isin(time_list)]
    return df_res


In [3]:
df_res = Read_Data()
assert df_res.shape==(3359,6)

In [4]:
assert list(df_res.columns) == ['reservation_date', 'reservation_time', 'reservation_party_size',
                                'reservation_booked_date', 'datetime_booked', 'reservation_datetime']

In [5]:
assert np.isclose(df_res.datetime_booked.dt.hour.mean(), 13.55969)

## Problem 2 

In this next part, we will write two functions to understand basic patterns in the data.

The first function takes as input one of the eight reservation time slots that we are considering as a datetime time and it outputs the day of the week (as a string) with the smallest average party size of all reservation made for the given inputted time slot.

In [6]:
import warnings
warnings.filterwarnings("ignore")

def Get_Avg_Party_Size(res_time):
    
    smallest_dow = None
    df_res = Read_Data()
    # YOUR CODE HERE
#     raise NotImplementedError()

    weekday_transfer = {
    0:'Monday',
    1:'Tuesday',
    2:'Wednesday',
    3:'Thursday',
    4:'Friday',
    5:'Saturday',
    6:'Sunday'
    }
    
    tmp_df = df_res[df_res['reservation_datetime'].dt.time == res_time]
    tmp_df['week'] = tmp_df['reservation_datetime'].dt.dayofweek
    tmp_df = tmp_df.groupby('week')['reservation_party_size'].mean()
    week = tmp_df.idxmin()
    return weekday_transfer[week]

In [7]:
res_time = datetime.time(hour = 17, minute=30)
assert Get_Avg_Party_Size(res_time)=="Sunday"

In [8]:
res_time = datetime.time(hour = 21, minute=0)
assert Get_Avg_Party_Size(res_time)=="Tuesday"

## Problem 3
The second function I would like you to write takes as input a parameter called num_days (you may assume this is an integer), and it returns the fraction of reservations made by parties size 1/2 that are made at most num_days in advance, and then also the same fraction but just for parties of 3/4.

**NOTE: The number of days in advance that a reservation is made is (reservation_datetime - datetime_booked).**

In [9]:
def Booked_in_Advance(num_days):
    
    df_res = Read_Data()
    num_days = datetime.timedelta(days=num_days)
    party_one_two = None
    party_three_four = None
    # YOUR CODE HERE
        
    df_res['state'] = df_res.apply(lambda x: 1 if (x['reservation_datetime'] - x['datetime_booked']).days < num_days.days else 0, axis=1)
    df_state = df_res[df_res['state'] == 1]
    
    part_size_all_12 = df_state[(df_state['reservation_party_size']==1) | (df_state['reservation_party_size']==2)]
    
    part_size_all_34 = df_state[(df_state['reservation_party_size']==3) | (df_state['reservation_party_size']==4)]
    
    party_one_two = len(part_size_all_12)/len(df_res[(df_res['reservation_party_size']==1) | (df_res['reservation_party_size']==2)])
    
    party_three_four = len(part_size_all_34)/len(df_res[(df_res['reservation_party_size']==3) | (df_res['reservation_party_size']==4)])
    
    return party_one_two, party_three_four


In [10]:
result = Booked_in_Advance(7)
assert np.isclose(result[0],0.27188)

In [11]:
result = Booked_in_Advance(100)
assert np.isclose(result[1],0.860534)

## Problem 4 

Let's assume that there are two service periods:

- Service Period 1 consists of time slots 17:30, 17:45, 18:00, 18:15
- Service Period 2 consiss of time slots 20:45, 21:00, 21:15, 21:30

The next function takes as input res_time, which is one of the eight times above as a datetime time.  Given the inputted time, you will exclusively focus this analysis on either Service Period 1 or 2. For example, if the inputted res_time is 17:45, you should only consider reservations made during Service Period 1. For each day, find the first reservation that books at res_time (datetime_booked tells you the order in which reservations were scheduled).  Within the given Service Period of interest, compute how many reservations have already been booked for the given day before this reservation for res_time was made.  If $k-1$ reservations have already been made, this means that res_time was first booked as the $k$-th reservation.  We will say the "rank" of reservation time res_time on this day was $k$. So, if res_time was the first time booked on a particular day for the relevant Service Period, its rank on this day is 1.  Return the average rank of res_time across all days in which a reservation for res_time was booked.

**Notice:** If there are two ranks which have the same booking time, they will have the same rank. For example, in the following table, both the first and second table share Rank 2. 

| reservation_dat | reservation_time | datetime_booked | rank |
|-----------------|------------------|-----------------|------|
|     9/9/17      |    18:15:00      |2017-08-15 10:03:00| 1|
|     9/9/17      |    18:00:00      |2017-08-15 10:04:00| 2|
|     9/9/17      |    17:45:00      |2017-08-15 10:04:00| 2|
|     9/9/17      |    17:30:00      |2017-08-15 10:05:00| 3|

In [12]:
def Get_Avg_Rank(res_time):

    df_res = Read_Data()
    avg_rank=None
    # YOUR CODE HERE
    
    time_1 = [
        datetime.time(hour = 17, minute=30),
        datetime.time(hour = 17, minute=45),
        datetime.time(hour = 18, minute=0),
        datetime.time(hour = 18, minute=15)
                ]
    
    time_2 = [
        datetime.time(hour = 20, minute=45),
        datetime.time(hour = 21, minute=0),
        datetime.time(hour = 21, minute=15),
        datetime.time(hour = 21, minute=30)
                ]
    
    if res_time in time_1:
        df_res_time = df_res[df_res['reservation_datetime'].dt.time.isin(time_1)]
    else:
        df_res_time = df_res[df_res['reservation_datetime'].dt.time.isin(time_2)]
    
    df_res_time = df_res_time[['reservation_datetime', 'datetime_booked', 'reservation_time']]
    df_res_time['date_booked'] = df_res_time['datetime_booked'].dt.strftime('%Y-%m-%d')
    df_res_time.sort_values(['datetime_booked'], inplace=True)
    
    # For each day, find the first reservation that books at res_time 
    # (datetime_booked tells you the order in which reservations were scheduled). 
    tmp_df = df_res_time[df_res_time['reservation_datetime'].dt.time == res_time]
    tmp_df['date_booked'] = tmp_df['datetime_booked'].dt.strftime('%Y-%m-%d')
    tmp_df['reservation_date'] = tmp_df['reservation_datetime']
    tmp_df['reservation_date'] = tmp_df['reservation_date'].dt.strftime('%Y-%m-%d')
    tmp_df.drop_duplicates(['reservation_date'], inplace=True)
    
    df_res_time['reservation_date'] = df_res_time['reservation_datetime'].dt.strftime('%Y-%m-%d')
    
    rank_list = []
    for item in tmp_df.to_dict('records'):
        datetime_booked = item['datetime_booked']
        date_booked = item['date_booked']
        reservation_datetime = item['reservation_datetime']
        reservation_date = item['reservation_date']
        
        datetime_booked_df = df_res_time[
            (df_res_time['datetime_booked'] < datetime_booked) 
          & (df_res_time['reservation_date'] == reservation_date)
        ]
        
        datetime_booked_list = datetime_booked_df['datetime_booked'].to_list()
        before_num = len(datetime_booked_list)
#         before_num = len(set(datetime_booked_list))
        
        res_time_num = before_num + 1
        rank_list.append(res_time_num)
    
    avg_rank = sum(rank_list)/len(rank_list)
    return avg_rank

In [13]:
res_time = datetime.time(hour = 21, minute=30)
assert np.isclose(Get_Avg_Rank(res_time),3.9035)

In [14]:
res_time = datetime.time(hour = 20, minute=45)
assert np.isclose(Get_Avg_Rank(res_time),1.45714)

In [15]:
res_time = datetime.time(hour = 17, minute=30)
assert np.isclose(Get_Avg_Rank(res_time),3.17690)

In [16]:
res_time = datetime.time(hour = 17, minute=45)
assert np.isclose(Get_Avg_Rank(res_time),2.77316)